In [2]:
import numpy as np
import pandas as pd

credits = pd.read_csv('../data/credits.csv')
keywords = pd.read_csv('../data/keywords.csv')
metadata = pd.read_csv('../data/movies_metadata.csv')

C:\Users\szymo\AppData\Local\Temp\ipykernel_2000\3065788958.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('../data/movies_metadata.csv')


In [3]:
metadata = metadata[['adult', 'genres', 'id', 'overview', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count']]

In [4]:
metadata.dropna(inplace=True)

In [5]:
metadata.drop_duplicates(inplace=True)

In [6]:
credits.drop_duplicates(inplace=True)

In [7]:
keywords.drop_duplicates(inplace=True)

In [8]:
metadata['id'] = metadata['id'].apply(lambda x : int(x))
metadata['popularity'] = metadata['popularity'].apply(lambda x : float(x))
metadata['vote_average'] = metadata['vote_average'].apply(lambda x : float(x))
metadata['vote_count'] = metadata['vote_count'].apply(lambda x : int(x))

In [9]:
merged = pd.merge(pd.merge(metadata, keywords, on='id'), credits, on='id')

In [10]:
import ast
from ast import literal_eval
# literal eval convert any string to its actual type

def convert(obj):

    L = []

    for i in literal_eval(obj):
        L.append(i['name'])

    return L

In [11]:
merged['genres'] = merged['genres'].apply(convert)
merged['keywords'] = merged['keywords'].apply(convert)

In [12]:
def convert_cast(obj):

    L = []
    counter = 0

    for i in literal_eval(obj):
        if counter != 5:
            L.append(i['name'])
            counter += 1

    return L

In [13]:
merged['cast'] = merged['cast'].apply(convert_cast)

In [14]:
def convert_crew(obj):

    for i in literal_eval(obj):
        if i['job'] == 'Director':
            return i['name']

In [15]:
merged['crew'] = merged['crew'].apply(convert_crew)

In [16]:
merged.dropna(inplace=True)

In [17]:
test_duplicate = merged.drop(columns=['genres', 'cast', 'keywords'])

In [18]:
merged['overview'] = merged['overview'].apply(lambda x: x.split())

In [19]:
merged['crew'] = merged['crew'].apply(lambda x: [x])

In [20]:
merged['tags'] = merged['overview'] + merged['genres'] + merged['keywords'] + merged['cast'] + merged['crew']

In [21]:
merged['tags'] = merged['tags'].apply(lambda x: [i.replace(" ", "") for i in x])

In [22]:
merged['tags'] = merged['tags'].apply(lambda x: " ".join(x))

In [23]:
merged['tags'] = merged['tags'].apply(lambda x : x.lower())

In [24]:
merged['release_date'] = merged['release_date'].apply(lambda x: int(x.split("-")[0]))

In [25]:
merged.reset_index(inplace=True)

In [26]:
age = []

for i in range(merged.shape[0]):
    if merged['release_date'][i] < 2001:
        age.append('old')
    else:
        age.append('new')

merged['age'] = age

In [27]:
review = []

for i in range(merged.shape[0]):
    if merged['vote_average'][i] <= 4:
        review.append('bad')
    elif merged['vote_average'][i] >= 5 and merged['vote_average'][i] <= 7:
        review.append('average')
    else:
        review.append('good')

merged['review'] = review

In [28]:
merged['final_tag'] = merged['tags'] + " " + merged['age'] + " " + merged['review']

In [29]:
def format(obj):

    obj = obj.split()
    obj = obj[:-2]

    obj = " ".join(obj)

    return obj

In [30]:
merged['final_tag'] = merged['final_tag'].apply(format)

In [31]:
merged.to_csv('../data/merged.csv')